# First load log file into the notebook

In [ ]:
pip install flatten_json


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Convert log file into json file


In [ ]:
# read lines from the nominations.log file and convert each json line into 
# a dictionary with all of the key value pairs within the json line
nominations = [json.loads(line)
        for line in open('nominations.log', 'r+')]
# convert json dictionaries back into strings 
nominations =  [json.dumps(element)
        for element in nominations]
# write all of the data within the nominations list into the nominations.json file
with open('nominations.json', 'w+') as convert_file:
  # add new line in between each json line when writing to the file
  convert_file.write("\n".join(nominations))

# Print Json file and convert into data frame

In [ ]:
import pandas as pd
import json
import ast
import warnings
from flatten_json import flatten
import numpy as np
import math
warnings.filterwarnings('ignore')
# opens log file
# turn string in file into json formatted string
data = [json.loads(line)
        for line in open('nominations_test1.json', 'r')] 
# normalize data so that it converts all of teh key value pairs into columns with values
# All missing or empty values are inputted as into np.nan
multiple_nominations = pd.json_normalize(data)
multiple_nominations

,nominationId,verb,path,country,version,track,album,remix,shareable,out.metadata.locales,...,out.metadata.customSettings.enableAmplitude.callterminated.failure,out.metadata.customSettings.enableAmplitude.refreshtoken.success,out.metadata.customSettings.enableAmplitude.refreshtoken.failure,out.metadata.customSettings.enableAmplitude.inboundoffered.success,out.metadata.customSettings.enableAmplitude.inboundoffered.failure,out.metadata.customSettings.enableAmplitude.inboundterminated.success,out.metadata.customSettings.enableAmplitude.inboundterminated.failure,in.parameters.sec-fetch-user,in.parameters.upgrade-insecure-requests,in.parameters.cache-control
0,aae6313e-4350-4285-a14f-045860e04010,GET,/public/v2.0/metadata/servicenow/test,us,v2.0,metadata,servicenow,test,False,[en],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9d59fbb0-fbbc-4edc-ac81-73a89447eab8,GET,/public/v2.0/metadata/servicenow,us,v2.0,metadata,servicenow,NaN,False,[en],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,e7d1c7c1-b1ea-466b-ab76-ea82e8d0acd4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6fc44a79-2fbc-445a-908f-888ee93e6f04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,61bdf010-1bf6-4db9-91e3-d96fee2a5720,POST,/public/v2.0/offered/salesforce,us,v2.0,offered,salesforce,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,f6d103fd-a88e-4470-8b4b-2c188fff115a,POST,/public/v2.0/terminated/salesforce,us,v2.0,terminated,salesforce,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,e78dd918-f8fa-4cce-a462-24232d01b093,POST,/public/v2.0/offered/zendesk,us,v2.0,offered,zendesk,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,900f202f-edf8-4b14-bff0-be7881a22496,GET,/public/v2.0/terminated/zendesk,us,v2.0,terminated,zendesk,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,72f4b0a6-3629-4899-921e-82b75660d753,POST,/public/v2.0/offered/netsuite,us,v2.0,offered,netsuite,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,732b9d8f-ccbb-44ab-9c0e-855db038e69f,POST,/public/v2.0/terminated/netsuite,us,v2.0,terminated,netsuite,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Add input and output label columns


In [ ]:
# add the input label and output label at the first and second column of the dataframe
multiple_nominations.insert(0, 'input_label', "")
multiple_nominations.insert(1, 'output_label', "")

In [ ]:
# turn all nan values into 0 for easier dataframe manipulation
multiple_nominations["out.records"] = multiple_nominations["out.records"].fillna(0)

# go through each row in data frame and add input/output label
for index, row in multiple_nominations.iterrows():
  # get input label
  multiple_nominations['input_label'][index] = str(row["customer"]) + "_" + str(row["album"]) + "_" + str(row["remix"]) + "_" + str(row["track"]) + "_" + str(row["in.parameters.query"])
  # get output label
  if (row["out.records"] == 0):
    multiple_nominations['output_label'][index] = "nan" + "_" + str(row["out.type"])
  else:
    multiple_nominations['output_label'][index] = str(len(row["out.records"])) + "_" + str(row["out.type"])
  if (pd.isna(row['out.warning'])):
    multiple_nominations['output_label'][index] = multiple_nominations['output_label'][index] + ""
  else:
    multiple_nominations['output_label'][index] = multiple_nominations['output_label'][index] + "_" + "warningexists"
# rename input/output with formulas for construction
multiple_nominations.rename(columns={'input_label': 'inputlabel : $.in.parameters.customer_$.album_$.remix_$.track_$.in.parameters.query', 'output_label': 'outputlabel: $.out.records.length_$.out.type_$.out.commands.~*_[?$.out.warning exists]warningexists'}, inplace=True)
multiple_nominations

,inputlabel : $.in.parameters.customer_$.album_$.remix_$.track_$.in.parameters.query,outputlabel: $.out.records.length_$.out.type_$.out.commands.~*_[?$.out.warning exists]warningexists,nominationId,verb,path,country,version,track,album,remix,...,out.metadata.customSettings.enableAmplitude.callterminated.failure,out.metadata.customSettings.enableAmplitude.refreshtoken.success,out.metadata.customSettings.enableAmplitude.refreshtoken.failure,out.metadata.customSettings.enableAmplitude.inboundoffered.success,out.metadata.customSettings.enableAmplitude.inboundoffered.failure,out.metadata.customSettings.enableAmplitude.inboundterminated.success,out.metadata.customSettings.enableAmplitude.inboundterminated.failure,in.parameters.sec-fetch-user,in.parameters.upgrade-insecure-requests,in.parameters.cache-control
0,nan_servicenow_test_metadata_nan,nan_nan,aae6313e-4350-4285-a14f-045860e04010,GET,/public/v2.0/metadata/servicenow/test,us,v2.0,metadata,servicenow,test,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,nan_servicenow_nan_metadata_nan,nan_nan,9d59fbb0-fbbc-4edc-ac81-73a89447eab8,GET,/public/v2.0/metadata/servicenow,us,v2.0,metadata,servicenow,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,nan_nan_nan_nan_nan,nan_nan,e7d1c7c1-b1ea-466b-ab76-ea82e8d0acd4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,nan_nan_nan_nan_nan,nan_nan,6fc44a79-2fbc-445a-908f-888ee93e6f04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,integrationautoma_salesforce_nan_offered_66927...,4_nan,61bdf010-1bf6-4db9-91e3-d96fee2a5720,POST,/public/v2.0/offered/salesforce,us,v2.0,offered,salesforce,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,integrationautoma_salesforce_nan_terminated_66...,nan_phonecall,f6d103fd-a88e-4470-8b4b-2c188fff115a,POST,/public/v2.0/terminated/salesforce,us,v2.0,terminated,salesforce,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,integrationautoma_zendesk_nan_offered_rajesh prod,2_nan,e78dd918-f8fa-4cce-a462-24232d01b093,POST,/public/v2.0/offered/zendesk,us,v2.0,offered,zendesk,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,integrationautoma_zendesk_nan_terminated_20966...,nan_nan,900f202f-edf8-4b14-bff0-be7881a22496,GET,/public/v2.0/terminated/zendesk,us,v2.0,terminated,zendesk,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8x8_netsuite_nan_offered_1 408 642-6156,4_nan,72f4b0a6-3629-4899-921e-82b75660d753,POST,/public/v2.0/offered/netsuite,us,v2.0,offered,netsuite,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,8x8_netsuite_nan_terminated_7620236468,nan_phonecall_warningexists,732b9d8f-ccbb-44ab-9c0e-855db038e69f,POST,/public/v2.0/terminated/netsuite,us,v2.0,terminated,netsuite,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# TEST

In [ ]:
# adding duplicate rows to test the drop feature
multiple_nominations = multiple_nominations.append(multiple_nominations.iloc[1], ignore_index=True)
multiple_nominations = multiple_nominations.append(multiple_nominations.iloc[3], ignore_index=True)
multiple_nominations = multiple_nominations.append(multiple_nominations.iloc[6], ignore_index=True)
multiple_nominations

,inputlabel : $.in.parameters.customer_$.album_$.remix_$.track_$.in.parameters.query,outputlabel: $.out.records.length_$.out.type_$.out.commands.~*_[?$.out.warning exists]warningexists,nominationId,verb,path,country,version,track,album,remix,...,out.metadata.customSettings.enableAmplitude.callterminated.failure,out.metadata.customSettings.enableAmplitude.refreshtoken.success,out.metadata.customSettings.enableAmplitude.refreshtoken.failure,out.metadata.customSettings.enableAmplitude.inboundoffered.success,out.metadata.customSettings.enableAmplitude.inboundoffered.failure,out.metadata.customSettings.enableAmplitude.inboundterminated.success,out.metadata.customSettings.enableAmplitude.inboundterminated.failure,in.parameters.sec-fetch-user,in.parameters.upgrade-insecure-requests,in.parameters.cache-control
0,nan_servicenow_test_metadata_nan,nan_nan,aae6313e-4350-4285-a14f-045860e04010,GET,/public/v2.0/metadata/servicenow/test,us,v2.0,metadata,servicenow,test,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,nan_servicenow_nan_metadata_nan,nan_nan,9d59fbb0-fbbc-4edc-ac81-73a89447eab8,GET,/public/v2.0/metadata/servicenow,us,v2.0,metadata,servicenow,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,nan_nan_nan_nan_nan,nan_nan,e7d1c7c1-b1ea-466b-ab76-ea82e8d0acd4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,integrationautoma_salesforce_nan_offered_66927...,4_nan,61bdf010-1bf6-4db9-91e3-d96fee2a5720,POST,/public/v2.0/offered/salesforce,us,v2.0,offered,salesforce,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,integrationautoma_salesforce_nan_terminated_66...,nan_phonecall,f6d103fd-a88e-4470-8b4b-2c188fff115a,POST,/public/v2.0/terminated/salesforce,us,v2.0,terminated,salesforce,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,integrationautoma_zendesk_nan_offered_rajesh prod,2_nan,e78dd918-f8fa-4cce-a462-24232d01b093,POST,/public/v2.0/offered/zendesk,us,v2.0,offered,zendesk,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,integrationautoma_zendesk_nan_terminated_20966...,nan_nan,900f202f-edf8-4b14-bff0-be7881a22496,GET,/public/v2.0/terminated/zendesk,us,v2.0,terminated,zendesk,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8x8_netsuite_nan_offered_1 408 642-6156,4_nan,72f4b0a6-3629-4899-921e-82b75660d753,POST,/public/v2.0/offered/netsuite,us,v2.0,offered,netsuite,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8x8_netsuite_nan_terminated_7620236468,nan_phonecall_warningexists,732b9d8f-ccbb-44ab-9c0e-855db038e69f,POST,/public/v2.0/terminated/netsuite,us,v2.0,terminated,netsuite,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,nan_salesforce_urbanoutfitters89401_metadata_nan,nan_nan,a5837fe1-aad7-4c4f-af7e-517d8d2a865d,GET,/public/v2.0/metadata/salesforce/urbanoutfitte...,us,v2.0,metadata,salesforce,urbanoutfitters89401,...,True,False,True,True,True,True,True,?1,1,max-age=0


# Drop non-unique rows

In [ ]:
multiple_nominations.drop_duplicates(subset=['inputlabel : $.in.parameters.customer_$.album_$.remix_$.track_$.in.parameters.query'], inplace = True)
multiple_nominations

,inputlabel : $.in.parameters.customer_$.album_$.remix_$.track_$.in.parameters.query,outputlabel: $.out.records.length_$.out.type_$.out.commands.~*_[?$.out.warning exists]warningexists,nominationId,verb,path,country,version,track,album,remix,...,out.metadata.customSettings.enableAmplitude.callterminated.failure,out.metadata.customSettings.enableAmplitude.refreshtoken.success,out.metadata.customSettings.enableAmplitude.refreshtoken.failure,out.metadata.customSettings.enableAmplitude.inboundoffered.success,out.metadata.customSettings.enableAmplitude.inboundoffered.failure,out.metadata.customSettings.enableAmplitude.inboundterminated.success,out.metadata.customSettings.enableAmplitude.inboundterminated.failure,in.parameters.sec-fetch-user,in.parameters.upgrade-insecure-requests,in.parameters.cache-control
0,nan_servicenow_test_metadata_nan,nan_nan,aae6313e-4350-4285-a14f-045860e04010,GET,/public/v2.0/metadata/servicenow/test,us,v2.0,metadata,servicenow,test,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,nan_servicenow_nan_metadata_nan,nan_nan,9d59fbb0-fbbc-4edc-ac81-73a89447eab8,GET,/public/v2.0/metadata/servicenow,us,v2.0,metadata,servicenow,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,nan_nan_nan_nan_nan,nan_nan,e7d1c7c1-b1ea-466b-ab76-ea82e8d0acd4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,integrationautoma_salesforce_nan_offered_66927...,4_nan,61bdf010-1bf6-4db9-91e3-d96fee2a5720,POST,/public/v2.0/offered/salesforce,us,v2.0,offered,salesforce,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,integrationautoma_salesforce_nan_terminated_66...,nan_phonecall,f6d103fd-a88e-4470-8b4b-2c188fff115a,POST,/public/v2.0/terminated/salesforce,us,v2.0,terminated,salesforce,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,integrationautoma_zendesk_nan_offered_rajesh prod,2_nan,e78dd918-f8fa-4cce-a462-24232d01b093,POST,/public/v2.0/offered/zendesk,us,v2.0,offered,zendesk,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,integrationautoma_zendesk_nan_terminated_20966...,nan_nan,900f202f-edf8-4b14-bff0-be7881a22496,GET,/public/v2.0/terminated/zendesk,us,v2.0,terminated,zendesk,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8x8_netsuite_nan_offered_1 408 642-6156,4_nan,72f4b0a6-3629-4899-921e-82b75660d753,POST,/public/v2.0/offered/netsuite,us,v2.0,offered,netsuite,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8x8_netsuite_nan_terminated_7620236468,nan_phonecall_warningexists,732b9d8f-ccbb-44ab-9c0e-855db038e69f,POST,/public/v2.0/terminated/netsuite,us,v2.0,terminated,netsuite,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,nan_salesforce_urbanoutfitters89401_metadata_nan,nan_nan,a5837fe1-aad7-4c4f-af7e-517d8d2a865d,GET,/public/v2.0/metadata/salesforce/urbanoutfitte...,us,v2.0,metadata,salesforce,urbanoutfitters89401,...,True,False,True,True,True,True,True,?1,1,max-age=0
